In [ ]:
import torch
import numpy as np
import pandas as pd
import random
from random import shuffle
from pathlib import Path
import os

import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.utils import make_grid
from torchvision import transforms as T,datasets
import torchvision

import torch.nn as nn
import torch.nn.functional as F


In [ ]:
train_dir = Path("/content/drive/MyDrive/SPR/data/archive/training/training")
test_dir = Path("/content/drive/MyDrive/SPR/data/archive/validation/validation")

cols = ['Label','Latin Name', 'Common Name','Train Images', 'Validation Images']
labels = pd.read_csv("/content/drive/MyDrive/SPR/data/archive/monkey_labels.txt" , names = cols, skiprows=1)
labels
print(os.listdir(train_dir))



['n4', 'n7', 'n9', 'n0', 'n8', 'n6', 'n1', 'n3', 'n2', 'n5']


In [ ]:
print(os.listdir(test_dir))


['n0', 'n7', 'n8', 'n9', 'n6', 'n2', 'n1', 'n3', 'n4', 'n5']


In [ ]:
train_tranform = T.Compose([T.Resize(size=(224,224)),T.ToTensor()])
val_tranform = T.Compose([T.Resize(size=(224,224)),T.ToTensor()])



#data augmentation for the train and test
train_data = ImageFolder(train_dir,       
                    transform=train_tranform)
val_data = ImageFolder(test_dir,
                    transform=val_tranform)




In [ ]:
#dataloader
batch_size = 8
train_dataloader = DataLoader(train_data , batch_size , shuffle = True , num_workers = 2 , pin_memory=True)
val_dataloader = DataLoader(val_data , batch_size , num_workers=2,pin_memory=True)


In [ ]:
image , label = val_data[50]
print(image.shape , label)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.Size([3, 224, 224]) 1


In [ ]:
from torch.nn.modules.activation import ReLU
#define a model
#input_dim = 3x224x224
class custom_model(nn.Module):
  def __init__(self):
    super(custom_model,self).__init__()
    self.model = nn.Sequential(
        nn.Conv2d(3,8,kernel_size=5,stride=1, padding=0), # 8x220x220
        nn.ReLU(),
        nn.Conv2d(8,16,kernel_size=5,stride=1,padding=0), # 16x216x216
        nn.ReLU(),
        nn.MaxPool2d(2,2), #16x108x108
        nn.Conv2d(16,32,kernel_size = 5,stride=1,padding=0), #32x104x104
        nn.ReLU(),
        nn.MaxPool2d(2,2), # 32x52x52
        nn.Conv2d(32,64,kernel_size=5,stride=1,padding=0), # 64x48x48
        nn.ReLU(),
        nn.Conv2d(64,128,kernel_size=5,stride=1,padding=0), #128x44x44
        nn.MaxPool2d(2,2), # 128x22x22
        nn.Flatten(), #(128*22*22)x1
        nn.Linear(128*22*22,1024),
        nn.ReLU(),
        nn.Linear(1024,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )

  def forward(self,x):
    return self.model(x)


In [ ]:
model = custom_model()
model

custom_model(
  (model): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (9): ReLU()
    (10): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=61952, out_features=1024, bias=True)
    (14): ReLU()
    (15): Linear(in_features=1024, out_features=512, bias=True)
    (16): ReLU()
    (17): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [ ]:
#train the network
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

learning_rate = 0.001
model = model.to(device)
cost_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

total_steps = len(train_data)



In [ ]:
# def evaluate(model,test_loader):
#   model.eval()
#   outputs = [model.validation_step(batch) for batch in test_loader]
#   return model.validation_epoch_end(outputs)

  

num_epochs = 5
for epoch in range(num_epochs):
  model.train()
  history = []
  train_losses = []
  for batch in train_dataloader:
    images , labels = batch
    outputs = model(images)
    losses = cost_function(outputs , labels)
    train_losses.append(losses)
    losses.backward()
    optimizer.step()
    optimizer.zero_grad()
  # if (i+1) % 400 == 0:
  print("Epoch [{}/{}]],Loss:{:.4f}".format(epoch+1, num_epochs, losses.item()))
    # result = evaluate(model, val_dataloader)
    # result['train_loss'] = torch.stack(train_losses).mean().item()
    # model.epoch_end(epoch, result)
    # history.append(result)

Epoch [1/5]],Loss:2.4068
Epoch [2/5]],Loss:2.3043
Epoch [3/5]],Loss:2.2521
Epoch [4/5]],Loss:2.3588


In [ ]:
# validate the results 
with torch.no_grad():
  total , correct = 0 , 0 
  for batch in val_dataloader:
    model.eval()
    # print(batch[1])
    images , labels = batch
    # print(image.shape , labels)
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    _,predicted = torch.max(output.data,1)
    # print(predicted)
    # print(labels)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    print("Correct :",correct)
  print("Accuracy of model: {} %".format(100*correct/total))

    


Correct : 0
Correct : 0
Correct : 0
Correct : 0
Correct : 0
Correct : 0
Correct : 0
Correct : 0
Correct : 0
Correct : 0
Correct : 7
Correct : 15
Correct : 23
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Correct : 30
Accuracy of model: 11.029411764705882 %


In [ ]:
#tranfer learning

pre_trained = torchvision.models.resnet18(pretrained=True)
# freeze all the layer so that only fc layer can be retrained.Do not compute gradients 
for param in pre_trained.parameters():
    param.requires_grad = False

num_ftrs = pre_trained.fc.in_features
pre_trained.fc = nn.Linear(num_ftrs, 10)

pre_trained = pre_trained.to(device)
criterion = nn.CrossEntropyLoss()

#optimize only the last layer
optimizer_conv = optim.SGD(pre_trained.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)


In [ ]:
# model_conv = train_model(model_conv, criterion, optimizer_conv,
#                          exp_lr_scheduler, num_epochs=25)
cost_function = nn.CrossEntropyLoss()

num_epochs = 5
for epoch in range(num_epochs):
  pre_trained.train()
  history = []
  train_losses = []
  for batch in train_dataloader:
    images , labels = batch
    outputs = pre_trained(images)
    losses = cost_function(outputs , labels)
    train_losses.append(losses)
    losses.backward()
    optimizer_conv.step()
    optimizer_conv.zero_grad()
  # if (i+1) % 400 == 0:
  print("Epoch [{}/{}]],Loss:{:.4f}".format(epoch+1, num_epochs, losses.item()))


Epoch [1/5]],Loss:1.5748
Epoch [2/5]],Loss:2.0805
Epoch [3/5]],Loss:2.7689
Epoch [4/5]],Loss:2.3695
Epoch [5/5]],Loss:2.5182


In [ ]:
with torch.no_grad():
  total , correct = 0 , 0 
  for batch in val_dataloader:
    pre_trained.eval()
    # print(batch[1])
    images , labels = batch
    # print(image.shape , labels)
    images = images.to(device)
    labels = labels.to(device)
    output = pre_trained(images)
    _,predicted = torch.max(output.data,1)
    # print(predicted)
    # print(labels)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    print("Correct :",correct)
  print("Accuracy of pre_trained model: {} %".format(100*correct/total))


Correct : 7
Correct : 15
Correct : 23
Correct : 31
Correct : 39
Correct : 47
Correct : 54
Correct : 62
Correct : 70
Correct : 78
Correct : 86
Correct : 94
Correct : 102
Correct : 110
Correct : 118
Correct : 126
Correct : 133
Correct : 141
Correct : 149
Correct : 156
Correct : 163
Correct : 171
Correct : 179
Correct : 186
Correct : 194
Correct : 202
Correct : 210
Correct : 218
Correct : 226
Correct : 234
Correct : 242
Correct : 250
Correct : 257
Correct : 265
Accuracy of pre_trained model: 97.42647058823529 %


In [ ]:
#finetune the network
finetune_model = models.resnet50(pretrained=True)
num_ftrs = finetune_model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
finetune_model.fc = nn.Linear(num_ftrs, 10)

finetune_model = finetune_model.to(device)

cost_function = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(finetune_model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
num_epochs = 2
for epoch in range(num_epochs):
  finetune_model.train()
  history = []
  train_losses = []
  for batch in train_dataloader:
    images , labels = batch
    outputs = finetune_model(images)
    losses = cost_function(outputs , labels)
    train_losses.append(losses)
    losses.backward()
    optimizer_ft.step()
    optimizer_ft.zero_grad()
  # if (i+1) % 400 == 0:
  print("Epoch [{}/{}]],Loss:{:.4f}".format(epoch+1, num_epochs, losses.item()))


Epoch [1/2]],Loss:3.1401
Epoch [2/2]],Loss:2.1549


In [26]:
with torch.no_grad():
  total , correct = 0 , 0 
  for batch in val_dataloader:
    finetune_model.eval()
    # print(batch[1])
    images , labels = batch
    # print(image.shape , labels)
    images = images.to(device)
    labels = labels.to(device)
    output = finetune_model(images)
    _,predicted = torch.max(output.data,1)
    # print(predicted)
    # print(labels)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    print("Correct :",correct)
  print("Accuracy of pre_trained model: {} %".format(100*correct/total))

Correct : 8
Correct : 16
Correct : 24
Correct : 32
Correct : 40
Correct : 48
Correct : 55
Correct : 63
Correct : 71
Correct : 79
Correct : 87
Correct : 95
Correct : 103
Correct : 111
Correct : 119
Correct : 127
Correct : 134
Correct : 142
Correct : 150
Correct : 158
Correct : 166
Correct : 174
Correct : 182
Correct : 190
Correct : 198
Correct : 206
Correct : 214
Correct : 222
Correct : 230
Correct : 238
Correct : 246
Correct : 254
Correct : 262
Correct : 270
Accuracy of pre_trained model: 99.26470588235294 %
